# Model Predictive Control
The general idea of figuring out what moves to make using optimization at each time step has become very popular due to the fact that a general version can be programmed and made very user friendly so that the intricacies of multivariable control can be handled by a single program.

In this notebook we will see how a single time step’s move trajectory is calculated. We’ll use the same linear system as was used for the controller proposed by E. B. Dahlin [1]

We start with a linear model of the system

$$G=\frac{1}{15s^2 + 8s + 1}$$

This differential equation could describe a physical mass-spring-damper mechanical system or a resistor-inductor-capacitor (RLC) circuit electrical system or other second-order linear systems. We can examine some details of this system. This is a stable and overdamped system. An instantaneous step up in the input signal u(t) from 0 to 1 will take about 20 seconds for our system to respond by moving from 0 to 0.95, which is almost 1. Adding a feedback controller can make the controlled “closed-loop” system to have a faster response to a step change in its input signal than the uncontrolled “open-loop” system. A model predictive controller is a powerfully type of feedback controller made possible by a reasonably accurate model of the system and fast computational processing.

#### Press ▶️ to plot the step response of a defined linear system.

In [1]:
import numpy
import scipy.signal
import scipy.optimize
import matplotlib.pyplot as plt
%matplotlib inline

G = scipy.signal.lti([1], [15, 8, 1])
plt.plot(*G.step())

print('Operation Complete')

ModuleNotFoundError: No module named 'numpy'

Our goal is to find out what manipulations must be made (changes to 
) in order to get the system to follow a specific desired trajectory (which we will call 
 for the reference trajectory). We will allow the controller to make a certain number of moves. This is called the control horizon, 
. We will the observe the effect of this set of moves (called a “move plan”) for time called the prediction horizon (
).


### Controller parameters

We define the controller parameters with a control horizon (the number of time steps that the controller predicts ahead in solving its optimization problem), a prediction horizon (the number of time steps ahead that we will simulate to evaluate controller performance), and a sampling rate, in this case 1 second. 

#### Press ▶️ to define control and prediction horizons, set the sampling rate, and generate time points for continuous and discrete representations over the prediction horizon.

In [ ]:
M = 10  # Control horizon
P = 20  # Prediction horizon
DeltaT = 1  # Sampling rate

tcontinuous = numpy.linspace(0, P*DeltaT, 1000)  # some closely spaced time points
tpredict = numpy.arange(0, P*DeltaT, DeltaT)   # discrete points at prediction horizon
print('Operation Complete')

To design a model predictive controller, we must have a desired behavior of the system. We will design by looking at step response behavior of the system. If we select a perfect step response as our reference, we will have difficulty evaluating the controller because that response is not physically achievable. A mass cannot be moved a displacement instantaneously without infinite energy. Instead of dealing in the infinite, let us use a reference trajectory for our system which is achievable: a first order system response. We choose a time constant tau_c for a desired first order response. Recall that a first order system reaches about 95% of its full step response in a period of time equal to about three times the time constant. If we set tau_c to 1, then we expect a response of our system to 95% in 3 seconds, which is fast compared to the 20 seconds we saw above for the open-loop system.

#### Press ▶️ to calculate the response curve based on a time constant and the exponential decay formula.

In [ ]:
tau_c = 1
r = 1 - numpy.exp(-tpredict/tau_c)
print('Operation Complete')

Now we build a model predictive controller. This controller solves an optimization problem before updating the input signal to the plant. This occurs once at every timestep at the selected speed for implementation of the controller. We will build out the ability to test the controller’s input signal, (t) here. 
We start with an array u(t) containing M elements as the controller predicts the next M timesteps when solving its optimization.

Next, we set the initial state x(t=0)=x_0 of the system to 0 as the predictions are for step responses away from an initial equilibrium at 0.

#### Press ▶️ to create an array of control inputs initialized to one for the specified control horizon.

In [ ]:
u = numpy.ones(M)
x0 = numpy.zeros(G.to_ss().A.shape[0])
print('Operation Complete')

Now, we define functions which will be useful for our controller and simulations. The extend function takes the input signal array (signal for each of the next M timesteps) as its argument and extends or concatenates the array with values of u at timestep M in the future as (u[-1], the final value of u returns 1) as the prediction will assume that the controller has turned off the input after M timesteps. We don’t store the extended u in a variable at this time, but instead return it as the returned value of the function.

#### Press ▶️ to extend the control input array, simulate the system response, and plot the predicted output over the prediction horizon.

In [ ]:
def extend(u):
    """We optimise the first M values of u but we need P values for prediction"""
    return numpy.concatenate([u, numpy.repeat(u[-1], P-M)])

plt.plot(tpredict, prediction(extend(u)))
print('Operation Complete')

The prediction function uses the linear system simulation function “lsim” to predict the response of the linear system, in this case, a controlled “closed loop” system to the extended input signal and the initial state of the system.

In [ ]:
def prediction(u, t=tpredict, x0=x0):
    """Predict the effect of an input signal"""
    t, y, x = scipy.signal.lsim(G, u, t, X0=x0, interp=False)
    return y

#### Press ▶️ to define a function that computes the objective value by evaluating the squared error between the predicted and desired responses.

In [ ]:
def objective(u, x0=x0):
    """Calculate the sum of the square error for the cotnrol problem"""
    y = prediction(extend(u))
    return sum((r - y)**2)
print('Operation Complete')

#### Press ▶️ to obtain the value of the objective for our step input:

In [ ]:
objective(u)

Now we figure out a set of moves which will minimize our objective function as the result of an optimization problem. Literally, we finding the u signal to minimizing the objective function

#### Press ▶️ to get a set of moves which will minimise our objective function

In [ ]:
result = scipy.optimize.minimize(objective, u)
uopt = result.x
result.fun

#### Press ▶️ to resample the discrete output to continuous time, effectively working out the zero-order hold value.

In [ ]:
ucont = extend(uopt)[((tcontinuous-0.01)//DeltaT).astype(int)]
print('Operation Complete')

#### Press ▶️ to plot the move plan and the output. Notice that we are getting exactly the output we want at the sampling times. At this point we have effectively recovered the controller of [1].

In [ ]:
def plotoutput(ucont, uopt):
    plt.figure()
    plt.plot(tcontinuous, ucont)
    plt.xlim([0, DeltaT*(P+1)])
    plt.figure()
    plt.plot(tcontinuous, prediction(ucont, tcontinuous), label='Continuous response')
    plt.plot(tpredict, prediction(extend(uopt)), '-o', label='Optimized response')
    plt.plot(tpredict, r, label='Set point')
    plt.legend()

plotoutput(ucont, uopt)
print('Operation Complete')

One of the reasons for the popularity of MPC is how easy it is to change its behavior using weights in the objective function. Try using this definition instead of the simple one above and see if you can remove the ringing in the controller output.

#### Press ▶️ to modify the behavior of the MPC by adjusting weights in the objective function, aiming to eliminate ringing in the controller output.

In [ ]:
def objective(u, x0=x0):
    y = prediction(extend(u))
    umag = numpy.abs(u)
    constraintpenalty = sum(umag[umag > 2])
    movepenalty = sum(numpy.abs(numpy.diff(u)))
    strongfinish = numpy.abs(y[-1] - r[-1])
    return sum((r - y)**2) + 0*constraintpenalty + 0.1*movepenalty + 0*strongfinish

objective(u, x0=x0)

You should find that this controlled system has slow response, but no high frequency ringing.

So we have seen that the controller can decide what to use as its next input signa, u(1). To move forward with our feedback control system, we would write a loop to continue the process of optimization at each timepoint moving forward, taking the output of the system at that time point as the new initial condition.

[1] Dahlin, E. B., Designing and Tuning Digital Controllers, Instrum. Control Systems, 41 (6), 77  (1968).